In [1]:
import pandas as pd
import requests
import telebot
import yfinance as yf
from datetime import date, datetime
import datetime as dt
from workadays import workdays as wd
from time import sleep
import numpy as np

In [76]:
# Coleta selic anual direto pelo BC
def calcula_cdi(vencimento):
    url_selic = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json'
    selic = pd.read_json(url_selic)
    selic['data'] = pd.to_datetime(selic['data'], dayfirst=True)
    selic.set_index('data', inplace=True)
    selic = float(selic.iloc[-1].values)

    # Calcula a quantidade de dias úteis e o rendimento do CDI até o vencimento
    data_hoje = dt.date.today()
    vencimento = datetime.strptime(vencimento, "%Y-%m-%d")
    dias_uteis = wd.networkdays(data_hoje, vencimento, country='BR')
    cdi_operacao = round(((1 + selic / 100) ** (dias_uteis / 252) - 1) * 100, 2)
    return cdi_operacao


def coleta_opcoes(ativo, vencimento):
    # Coleta o preço do ativo com base no último fechamento
    preco_ativo = round(yf.download(ativo +'.SA', period='1d')['Adj Close'].iloc[-1], 2)
    
    # Coleta informações das opções
    url = f'https://opcoes.net.br/listaopcoes/completa?idAcao={ativo}&listarVencimentos=false&cotacoes=true&vencimentos={vencimento}'
    r = requests.get(url).json()
    x = ([ativo, vencimento, i[0].split('_')[0], i[2], i[3], i[5], i[8], i[9], i[10], i[11]] for i in r['data']['cotacoesOpcoes'])
    df = pd.DataFrame(x, columns=['ativo', 'vencimento', 'ticker', 'tipo', 'modelo', 'strike', 'preco', 'negocios', 'volume', 'data ult'])
    df['data ult'] = pd.to_datetime(df['data ult'], dayfirst=True).dt.date

        
    # Cria um dataframe somente com as PUTs
    df_put = df[df['tipo'] == 'PUT']
    df_put_op = df_put.copy()
    df_put_op.rename(columns={
        'ticker': 'ticker_put',
        'tipo': 'tipo_put',
        'modelo': 'modelo_put',
        'strike': 'strike_put', 
        'preco': 'preco_put',
        'negocios': 'negocios_put',
        'volume': 'volume_put',
        'data ult': 'data ult_put'
    }, inplace=True)

    # Cria um dataframe somente com as CALLs e renomeia colunas para diferenciar
    df_call = df[df['tipo'] == 'CALL']
    df_call_op = df_call.copy()
    df_call_op.drop(columns=['vencimento'], inplace=True)
    df_call_op.rename(columns={
        'ticker': 'ticker_call',
        'tipo': 'tipo_call',
        'modelo': 'modelo_call',
        'strike': 'strike_call', 
        'preco': 'preco_call',
        'negocios': 'negocios_call',
        'volume': 'volume_call',
        'data ult': 'data ult_call'
    }, inplace=True)

    # Cria um dataframe com as operações possíveis para cada PUT
    df = pd.merge(df_put_op, df_call_op, on='ativo', suffixes=('', '_call'))
    df['preco ativo'] = preco_ativo
    
    return df, df_put, df_call, preco_ativo


# Operação - Collar de Alta
def collar_alta(ativo, vencimento, quantidade = 1, volume_put = 0.01, negocios_put = 1, volume_call = 0.01, 
                negocios_call = 1, filtro_data=None, risco = 0.00, corretagem_variavel = 0.00, corretagem_ordem = 0.00):
    filtro_data = datetime.strptime(filtro_data, "%Y-%m-%d").date()
    # Coleta as opções disponíveis para o ativo com base no vencimento determinado
    df, df_put, df_call, preco_ativo = coleta_opcoes(ativo, vencimento)
    
    tx_b3 = 0.1340
    df['custo'] = round((preco_ativo + df['preco_put'] - df['preco_call']) * quantidade, 2)
    df['corretagem'] = round((df['custo'] * ((corretagem_variavel + tx_b3) / 100)) + 6 * corretagem_ordem, 2)
    df['corretagem %'] = round(df['corretagem'] / df['custo'] * 100, 2)
    df['risco %'] = float(risco)
    df['lucro minimo'] = round(df['strike_put'] * quantidade - df['custo'], 2)
    df['lucro min %'] = round(df['lucro minimo'] / df['custo'] * 100, 2)
    df['lucro maximo'] = round(df['strike_call'] * quantidade - df['custo'], 2)
    df['lucro max %'] = round(df['lucro maximo'] / df['custo'] * 100, 2)
    
    # Filtra o dataframe somente com as operações lucrativas
    df_op = df.copy()
    df_op = df_op[df_op['lucro min %'] >= df['corretagem %'] + df['risco %']]
    df_op = df_op[df_op['lucro maximo'] > 0]
    df_op = df_op[df_op['strike_put'] > preco_ativo]
    df_op = df_op[df_op['strike_call'] > df_op['strike_put']]
    df_op = df_op[df_op['volume_put'] >= volume_put]
    df_op = df_op[df_op['negocios_put'] >= negocios_put]
    df_op = df_op[df_op['volume_call'] >= volume_call]
    df_op = df_op[df_op['negocios_call'] >= negocios_call]
    df_op = df_op[df_op['data ult_put'] >= filtro_data]
    df_op = df_op[df_op['data ult_call'] >= filtro_data]
    df_op = df_op.sort_values(by='lucro min %',ascending=True)
    
    return df, df_put, df_call, df_op


# Operação - Collar de Baixa
def collar_baixa(ativo, vencimento, quantidade = 1, volume_put = 0.01, negocios_put = 1, volume_call = 0.01, 
                 negocios_call = 1, filtro_data=None, risco = 0.00, corretagem_variavel = 0.00, corretagem_ordem = 0.00):
    filtro_data = datetime.strptime(filtro_data, "%Y-%m-%d").date()
    # Coleta as opções disponíveis para o ativo com base no vencimento determinado
    df, df_put, df_call, preco_ativo = coleta_opcoes(ativo, vencimento)

    tx_b3 = 0.1340
    df['custo'] = round((preco_ativo + df['preco_put'] - df['preco_call']) * quantidade, 2)
    df['corretagem'] = round((df['custo'] * ((corretagem_variavel + tx_b3) / 100)) + 6 * corretagem_ordem, 2)
    df['corretagem %'] = round(df['corretagem'] / df['custo'] * 100, 2)
    df['risco %'] = float(risco)
    df['lucro minimo'] = round(df['strike_call'] * quantidade - df['custo'], 2)
    df['lucro min %'] = round(df['lucro minimo'] / df['custo'] * 100, 2)
    df['lucro maximo'] = round(df['strike_put'] * quantidade - df['custo'], 2)
    df['lucro max %'] = round(df['lucro maximo'] / df['custo'] * 100, 2)
    
    # Filtra o dataframe somente com as operações lucrativas
    df_op = df.copy()
    df_op = df_op[df_op['lucro min %'] >= df['corretagem %'] + df['risco %']]
    df_op = df_op[df_op['lucro maximo'] > 0]
    df_op = df_op[df_op['strike_put'] < preco_ativo + 2]
    df_op = df_op[df_op['strike_call'] < df_op['strike_put']]
    df_op = df_op[df_op['volume_put'] >= volume_put]
    df_op = df_op[df_op['negocios_put'] >= negocios_put]
    df_op = df_op[df_op['volume_call'] >= volume_call]
    df_op = df_op[df_op['negocios_call'] >= negocios_call]
    df_op = df_op[df_op['data ult_put'] >= filtro_data]
    df_op = df_op[df_op['data ult_call'] >= filtro_data]
    df_op = df_op.sort_values(by='lucro min %',ascending=True)
    
    return df, df_put, df_call, df_op


# Coleta as opções com todos os vencimentos
def opt_all(ativo):
    url = f'https://opcoes.net.br/listaopcoes/completa?idLista=ML&idAcao={ativo}&listarVencimentos=true&cotacoes=true'
    r = requests.get(url).json()
    vencimentos = [i['value'] for i in r['data']['vencimentos']]
    df = pd.concat([coleta_opcoes(ativo, vencimento) for vencimento in vencimentos])
    return df



In [77]:
ativo = 'BOVA11'
vencimento = '2024-03-15'       # YYYY-MM-DD
quantidade = 1                 # quantidade é igual para o ativo, put e call
cdi = calcula_cdi(vencimento)   # calcula o cdi da operação até o vencimento
volume_put = 0.01               # volume mínimo negociado em put
negocios_put = 1                # quantidade mínima de negócios em put
volume_call = 0.01              # volume mínimo negociado em call
negocios_call = 1               # quantidade mínima de negócios em call
filtro_data = '2024-02-23'      # retorna somente as opções negociadas após esse dia
risco = -1.45                  # risco máximo de perda. Pode ser qualquer valor ou vincular à variável cdi
corretagem_variavel = 0.00      # representa 0,50% de corretagem sobre o volume negociado
corretagem_ordem = 0.00         # no caso, corretagem gratuita

df, df_put_alta, df_call_alta, df_op_collar_alta = collar_alta(ativo, vencimento, quantidade, volume_put, negocios_put, volume_call, 
                                                               negocios_call, filtro_data, risco, corretagem_variavel, corretagem_ordem)

data_columns = ['vencimento', 'data ult_put', 'data ult_call']
for col in data_columns:
    df_op_collar_alta[col] = pd.to_datetime(df_op_collar_alta[col]).dt.strftime('%d/%m/%Y')

float_columns = df.select_dtypes(include=[np.float64]).columns
for col in float_columns:
    df_op_collar_alta[col] = df_op_collar_alta[col].apply(lambda x: str(x).replace('.', ',') if pd.notnull(x) else x)

df_op_collar_alta


[*********************100%%**********************]  1 of 1 completed
C:\Users\Robson\AppData\Local\Temp\ipykernel_25200\439202174.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data ult'] = pd.to_datetime(df['data ult'], dayfirst=True).dt.date


,ativo,vencimento,ticker_put,tipo_put,modelo_put,strike_put,preco_put,negocios_put,volume_put,data ult_put,...,data ult_call,preco ativo,custo,corretagem,corretagem %,risco %,lucro minimo,lucro min %,lucro maximo,lucro max %
2112,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,15","0,17","0,13","-1,45","-1,15","-0,89","14,85","11,5"
2113,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,15","0,17","0,13","-1,45","-1,15","-0,89","16,85","13,05"
2110,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,13","0,17","0,13","-1,45","-1,13","-0,88","10,87","8,42"
2111,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,14","0,17","0,13","-1,45","-1,14","-0,88","12,86","9,96"
2109,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,11","0,17","0,13","-1,45","-1,11","-0,86","8,89","6,89"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627,BOVA11,15/03/2024,BOVAO136,PUT,E,"136,0","7,72","71,0","193058,43",27/02/2024,...,27/02/2024,"127,8","135,47","0,18","0,13","-1,45","0,53","0,39","2,53","1,87"
2626,BOVA11,15/03/2024,BOVAO136,PUT,E,"136,0","7,72","71,0","193058,43",27/02/2024,...,27/02/2024,"127,8","135,45","0,18","0,13","-1,45","0,55","0,41","1,55","1,14"
2852,BOVA11,15/03/2024,BOVAO140,PUT,E,"140,0","11,55","20,0","111346,4",27/02/2024,...,27/02/2024,"127,8","139,34","0,19","0,14","-1,45","0,66","0,47","4,66","3,34"
2853,BOVA11,15/03/2024,BOVAO140,PUT,E,"140,0","11,55","20,0","111346,4",27/02/2024,...,27/02/2024,"127,8","139,34","0,19","0,14","-1,45","0,66","0,47","6,66","4,78"


In [78]:
pd.set_option('display.max_rows', None)

In [80]:
df_op_collar_alta

,ativo,vencimento,ticker_put,tipo_put,modelo_put,strike_put,preco_put,negocios_put,volume_put,data ult_put,...,data ult_call,preco ativo,custo,corretagem,corretagem %,risco %,lucro minimo,lucro min %,lucro maximo,lucro max %
2112,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,15","0,17","0,13","-1,45","-1,15","-0,89","14,85","11,5"
2113,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,15","0,17","0,13","-1,45","-1,15","-0,89","16,85","13,05"
2110,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,13","0,17","0,13","-1,45","-1,13","-0,88","10,87","8,42"
2111,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,14","0,17","0,13","-1,45","-1,14","-0,88","12,86","9,96"
2109,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,11","0,17","0,13","-1,45","-1,11","-0,86","8,89","6,89"
2108,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,09","0,17","0,13","-1,45","-1,09","-0,84","7,91","6,13"
2107,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","129,06","0,17","0,13","-1,45","-1,06","-0,82","6,94","5,38"
3001,BOVA11,15/03/2024,BOVAO144,PUT,E,"144,0","17,35","1,0","13880,0",26/02/2024,...,27/02/2024,"127,8","145,14","0,19","0,13","-1,45","-1,14","-0,79","0,86","0,59"
2106,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","128,95","0,17","0,13","-1,45","-0,95","-0,74","5,05","3,92"
2105,BOVA11,15/03/2024,BOVAO128,PUT,E,"128,0","1,36","1991,0","1341139,56",27/02/2024,...,27/02/2024,"127,8","128,84","0,17","0,13","-1,45","-0,84","-0,65","4,16","3,23"


In [70]:
df.loc[2115]


KeyError: 2115

In [72]:
url = f'https://opcoes.net.br/listaopcoes/completa?idAcao={ativo}&listarVencimentos=false&cotacoes=true&vencimentos={vencimento}'
r = requests.get(url).json()
x = ([ativo, vencimento, i[0].split('_')[0], i[2], i[3], i[5], i[8], i[9], i[10], i[11]] for i in r['data']['cotacoesOpcoes'])
df = pd.DataFrame(x, columns=['ativo', 'vencimento', 'ticker', 'tipo', 'modelo', 'strike', 'preco', 'negocios', 'volume', 'data ult'])
df

,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
0,BOVA11,2024-03-15,BOVAC12,CALL,E,121.0,7.51,30.0,156965.64,17:53:21
1,BOVA11,2024-03-15,BOVAC15,CALL,A,150.0,0.01,2.0,2.99,14/02/2024
2,BOVA11,2024-03-15,BOVAC50,CALL,A,50.0,77.58,1.0,7758.00,12:13:00
3,BOVA11,2024-03-15,BOVAC91,CALL,A,91.0,34.58,5.0,24207.50,19/02/2024
4,BOVA11,2024-03-15,BOVAC95,CALL,A,102.0,25.29,1.0,49821.30,22/02/2024
...,...,...,...,...,...,...,...,...,...,...
143,BOVA11,2024-03-15,BOVAO950,PUT,E,95.0,0.01,1.0,10.00,26/02/2024
144,BOVA11,2024-03-15,BOVAO960,PUT,E,96.0,0.01,6.0,205.00,21/02/2024
145,BOVA11,2024-03-15,BOVAO970,PUT,E,97.0,0.01,2.0,150.00,21/02/2024
146,BOVA11,2024-03-15,BOVAO980,PUT,E,98.0,0.01,8.0,820.00,21/02/2024


In [75]:
df.loc[df['ticker'] == 'BOVAC152']

,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
43,BOVA11,2024-03-15,BOVAC152,CALL,A,152.0,0.01,1.0,100.0,2024-02-07


In [74]:
df.loc[df['ticker'] == 'BOVAC158']

,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
46,BOVA11,2024-03-15,BOVAC158,CALL,A,158.0,0.02,1.0,2.0,2024-02-07


In [73]:
df['data ult'] = pd.to_datetime(df['data ult'], dayfirst=True).dt.date
df

C:\Users\Robson\AppData\Local\Temp\ipykernel_25200\3474172541.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['data ult'] = pd.to_datetime(df['data ult'], dayfirst=True).dt.date


,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
0,BOVA11,2024-03-15,BOVAC12,CALL,E,121.0,7.51,30.0,156965.64,2024-02-27
1,BOVA11,2024-03-15,BOVAC15,CALL,A,150.0,0.01,2.0,2.99,2024-02-14
2,BOVA11,2024-03-15,BOVAC50,CALL,A,50.0,77.58,1.0,7758.00,2024-02-27
3,BOVA11,2024-03-15,BOVAC91,CALL,A,91.0,34.58,5.0,24207.50,2024-02-19
4,BOVA11,2024-03-15,BOVAC95,CALL,A,102.0,25.29,1.0,49821.30,2024-02-22
...,...,...,...,...,...,...,...,...,...,...
143,BOVA11,2024-03-15,BOVAO950,PUT,E,95.0,0.01,1.0,10.00,2024-02-26
144,BOVA11,2024-03-15,BOVAO960,PUT,E,96.0,0.01,6.0,205.00,2024-02-21
145,BOVA11,2024-03-15,BOVAO970,PUT,E,97.0,0.01,2.0,150.00,2024-02-21
146,BOVA11,2024-03-15,BOVAO980,PUT,E,98.0,0.01,8.0,820.00,2024-02-21


In [98]:
df # retorna o dataframe completo com todas as operações possíveis, independente de lucro ou prejuízo. Calcula para cada put, todas as calls

,ativo,vencimento,ticker_put,tipo_put,modelo_put,strike_put,preco_put,negocios_put,volume_put,data ult_put,...,data ult_call,preco ativo,custo,corretagem,corretagem %,risco %,lucro minimo,lucro min %,lucro maximo,lucro max %
0,BOVA11,2024-03-15,BOVAO12,PUT,E,121.0,0.24,191.0,98575.33,2024-02-26,...,2024-02-26,127.74,1222.5,7.75,0.63,-1.45,-12.5,-1.02,-12.5,-1.02
1,BOVA11,2024-03-15,BOVAO12,PUT,E,121.0,0.24,191.0,98575.33,2024-02-26,...,2024-02-14,127.74,1279.7,8.11,0.63,-1.45,-69.7,-5.45,220.3,17.21
2,BOVA11,2024-03-15,BOVAO12,PUT,E,121.0,0.24,191.0,98575.33,2024-02-26,...,2024-02-26,127.74,522.8,3.31,0.63,-1.45,687.2,131.45,-22.8,-4.36
3,BOVA11,2024-03-15,BOVAO12,PUT,E,121.0,0.24,191.0,98575.33,2024-02-26,...,2024-02-19,127.74,934.0,5.92,0.63,-1.45,276.0,29.55,-24.0,-2.57
4,BOVA11,2024-03-15,BOVAO12,PUT,E,121.0,0.24,191.0,98575.33,2024-02-26,...,2024-02-22,127.74,1026.9,6.51,0.63,-1.45,183.1,17.83,-6.9,-0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5471,BOVA11,2024-03-15,BOVAO990,PUT,E,99.0,0.01,2.0,0.11,2024-02-26,...,2024-02-23,127.74,966.5,6.13,0.63,-1.45,23.5,2.43,-16.5,-1.71
5472,BOVA11,2024-03-15,BOVAO990,PUT,E,99.0,0.01,2.0,0.11,2024-02-26,...,None,127.74,NaN,NaN,NaN,-1.45,NaN,NaN,NaN,NaN
5473,BOVA11,2024-03-15,BOVAO990,PUT,E,99.0,0.01,2.0,0.11,2024-02-26,...,2024-02-26,127.74,981.5,6.22,0.63,-1.45,8.5,0.87,-11.5,-1.17
5474,BOVA11,2024-03-15,BOVAO990,PUT,E,99.0,0.01,2.0,0.11,2024-02-26,...,2024-02-21,127.74,991.0,6.28,0.63,-1.45,-1.0,-0.10,-11.0,-1.11


In [14]:
df_put_alta # retorna todas as puts disponíveis para o ativo

,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
74,BOVA11,2024-03-15,BOVAO12,PUT,E,121.0,0.14,323.0,46632.89,1900-01-01
75,BOVA11,2024-03-15,BOVAO15,PUT,E,150.0,21.65,1.0,21.65,1900-01-01
76,BOVA11,2024-03-15,BOVAO50,PUT,E,50.0,0.01,1.0,10.00,2024-01-18
77,BOVA11,2024-03-15,BOVAO91,PUT,E,91.0,0.01,1.0,500.00,2024-02-19
78,BOVA11,2024-03-15,BOVAO95,PUT,E,102.0,0.01,8.0,676.00,2024-02-26
...,...,...,...,...,...,...,...,...,...,...
143,BOVA11,2024-03-15,BOVAO950,PUT,E,95.0,0.01,1.0,10.00,2024-02-26
144,BOVA11,2024-03-15,BOVAO960,PUT,E,96.0,0.01,6.0,205.00,2024-02-21
145,BOVA11,2024-03-15,BOVAO970,PUT,E,97.0,0.01,2.0,150.00,2024-02-21
146,BOVA11,2024-03-15,BOVAO980,PUT,E,98.0,0.01,8.0,820.00,2024-02-21


In [33]:
df_call_alta # retorna todas as puts disponíveis para o ativo

,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
0,BOVA11,2024-03-15,BOVAC12,CALL,E,121.0,5.73,16.0,72796.46,2024-02-26
1,BOVA11,2024-03-15,BOVAC15,CALL,A,150.0,0.01,2.0,2.99,2024-02-14
2,BOVA11,2024-03-15,BOVAC50,CALL,A,50.0,75.70,2.0,71225.00,2024-02-26
3,BOVA11,2024-03-15,BOVAC91,CALL,A,91.0,34.58,5.0,24207.50,2024-02-19
4,BOVA11,2024-03-15,BOVAC95,CALL,A,102.0,25.29,1.0,49821.30,2024-02-22
...,...,...,...,...,...,...,...,...,...,...
69,BOVA11,2024-03-15,BOVAC950,CALL,A,95.0,31.10,1.0,3732.00,2024-02-23
70,BOVA11,2024-03-15,BOVAC960,CALL,A,96.0,NaN,NaN,NaN,None
71,BOVA11,2024-03-15,BOVAC970,CALL,A,97.0,29.60,1.0,148000.00,2024-02-26
72,BOVA11,2024-03-15,BOVAC980,CALL,A,98.0,28.65,2.0,85950.10,2024-02-21


In [93]:
#ENVIANDO MENSAGEM PARA TELEGRAM COM O RANKING
bot = telebot.TeleBot("6832366281:AAF8pcibmxZGWyPUeGHgHOT64qkMgiKGCQA")
grupo = "-1001803272086"

for index, row in df_op_collar_alta.iterrows():
    message = (
        f"🟢COLLAR DE ALTA {row['ativo']}\n"
        #f"ATIVO: {row['ativo']}\n"
        f"PREÇO {row['ativo']}: R$ {row['preco ativo']}\n"
        f"VENCIMENTO: {row['vencimento']}\n"
        f"PUT: {row['ticker_put']}\n"
        f"PREÇO PUT: R$ {row['preco_put']}\n"
        f"CALL: {row['ticker_call']}\n"
        f"PREÇO CALL: R$ {row['preco_call']}\n"
        f"RISCO: {row['lucro min %']}%\n"
        f"LUCRO MAX: {row['lucro max %']}%\n"
        "-----------------------------------"
    )
    
    # Enviando a mensagem para o canal
    bot.send_message(grupo, message, protect_content=True, message_thread_id=75)
    sleep(30)